In [15]:
import torch

model = torch.hub.load('ultralytics/yolov5', 'yolov5s')
model.conf = 0.25  # confidence threshold (0-1)
model.iou = 0.45  # NMS IoU threshold (0-1)
model.classes = 0 
model.agnostic_nms = True
# im = 'https://ultralytics.com/images/zidane.jpg'  # file, Path, PIL.Image, OpenCV, nparray, list
# results = model(im)  # inference
# results.print()  

def bbox_r(width, height, *xyxy):
    """" Calculates the relative bounding box from absolute pixel values. """
    bbox_left = min([xyxy[0].item(), xyxy[2].item()])
    bbox_top = min([xyxy[1].item(), xyxy[3].item()])
    bbox_w = abs(xyxy[0].item() - xyxy[2].item())
    bbox_h = abs(xyxy[1].item() - xyxy[3].item())
    x_c = (bbox_left + bbox_w / 2)
    y_c = (bbox_top + bbox_h / 2)
    w = bbox_w
    h = bbox_h
    return x_c, y_c, w, h


from round_utils.dataload import LoadVideo
import cv2

video_path = '/Users/duan/Documents/Yolov5-Deepsort-Fastreid/game_cut_demo.mp4'
video = LoadVideo(video_path,img_size=640,start=0,end=1000)

for img, ori_img, vid_cap in video:
    # 将img的维度转换为（384，640，3）
    img = img.transpose(1,2,0)
    print(img.shape)
    results = model(img)
    # results.print()
    # results.save()
    # results.show()

    detections = results.pandas().xyxy[0]
    for _, row in detections.iterrows():
        x1, y1, x2, y2 = row[['xmin', 'ymin', 'xmax', 'ymax']]
        # 将x1, y1, x2, y2的大小转换到到ori_img的大小
        x1 = int(x1 * ori_img.shape[1] / img.shape[1])
        y1 = int(y1 * ori_img.shape[0] / img.shape[0])
        x2 = int(x2 * ori_img.shape[1] / img.shape[1])
        y2 = int(y2 * ori_img.shape[0] / img.shape[0])
        print(bbox_r(ori_img.shape[1],ori_img.shape[0],[x1, y1, x2, y2]))
        # 将这个坐标框在ori_img上画出来
        cv2.rectangle(ori_img, (x1, y1), (x2, y2), (0, 0, 255), 2)
    
    # 写入图片
    cv2.imwrite('test.jpg',ori_img)

    print(ori_img.shape)
    break


Using cache found in /Users/duan/.cache/torch/hub/ultralytics_yolov5_master
YOLOv5 🚀 2023-11-25 Python-3.11.5 torch-2.1.1 CPU

Fusing layers... 
YOLOv5s summary: 213 layers, 7225885 parameters, 0 gradients, 16.4 GFLOPs
Adding AutoShape... 


(384, 640, 3)
(1384.5, 688.0, 123, 300)
(953.0, 954.0, 184, 184)
(595.0, 521.5, 90, 217)
(970.0, 382.0, 60, 136)
(736.0, 320.5, 36, 73)
(779.5, 324.5, 55, 69)
(556.0, 320.5, 52, 73)
(503.0, 322.5, 42, 61)
(1080, 1920, 3)
